Spare Terminal Window

In [ ]:
!

Installing Runtime Dependencies (Run every time the virtual machine is started)

In [ ]:
!apt install aria2
!pip install sharepy ffmpeg yt-dlp
!python -m pip install --upgrade pip setuptools wheel && python -m pip install lbry-libtorrent wget torf && apt install python3-libtorrent
!pip install git+https://github.com/simon-weber/gpsoauth@master
!pip3 install --upgrade urllib3

For downloading anything to drive from a DOWNLOAD LINK only

In [ ]:
link=r"{}".format(input("Enter link: "))
outputFolder=r"/content/drive/MyDrive/{}".format(input("Choose output folder on your drive: /"))
!wget "{link}"

For downloading a youtube video, a youtube playlist, or a single google drive video (works with unlisted YT videos and view-only GDrive videos) (will work with a lot of sites, see https://github.com/yt-dlp/yt-dlp )

In [ ]:
import subprocess
link=r"{}".format(input("Enter link: "))
outputFolder=r"/content/drive/MyDrive/{}".format(input("Choose output folder on your drive: /"))
command=r'yt-dlp "{}" --format-sort res:1080,ext:mp4:m4a -P "/content/temp/" -o "%(title)s.%(ext)s" --cookies cookies.txt'.format(link)
sout = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE).stdout
print(sout.read().decode())
!mv "/content/temp/*" "{outputFolder}"

For downloading files from teams (sharepoint/onedrive)\
Downloads them to /content/sites/TEAMNAME/FOLDERNAME/FILENAME.ext in your VM

In [ ]:
import sharepy
import os

s = sharepy.connect("alexuuni.sharepoint.com")
siteName = input("Enter Site (Team) Name: ")
site = "https://alexuuni.sharepoint.com/sites/{}".format(siteName)
library = input("Enter Library Name: ")

folder=input("Enter folder name (may be path) (Optional): ")

files = s.get("{}/_api/web/lists/GetByTitle('{}')/items?$select=FileLeafRef,FileRef"
              .format(site, library)).json()["d"]["results"]

def getFile(file):
    source = "https://" + s.site + file["FileRef"]
    dest = "." + file["FileRef"]
    folder = s.get("{}/_api/web/GetFolderByServerRelativeUrl('{}')" .format(site, file["FileRef"].replace("'","''"))).status_code == 200

    if not os.path.exists(dest):
        if folder:
            os.makedirs(dest)
        else:
            s.getfile(source, filename=dest)

for file in files:
  if (folder == ""):
    getFile(file["FileRef"])
  elif r"{}".format(file["FileRef"]).startswith(r'/sites/{}/{}/{}'.format(siteName,library,folder)):
    getFile(file)


For downloading videos from a google drive folder\
Downloads directly into google drive

In [ ]:
from yt_dlp import YoutubeDL as ydl
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

rootID = input("Enter Folder ID: ")
downloadPath = input("Enter Path to Output Folder in Drive: /")

def listFiles(folderID):
    return drive.ListFile({'q': "'{}' in parents".format(folderID)}).GetList()

acceptedFileList = []

def filterFiles(folderID):
  file_list = listFiles(folderID)
  for file in file_list:
    if 'video' in file["mimeType"]:
      file = {"name": file["title"].rsplit('.', 1)[0], "url": "https://drive.google.com/file/d/{}/view?usp=drive_link".format(file["id"])}
      acceptedFileList.append(file)
    elif file["mimeType"] == 'application/vnd.google-apps.folder':
      filterFiles(file["id"])


def downloadVideos():
  filterFiles(rootID)
  for f in acceptedFileList:
    print(f)
    !yt-dlp -P "/content/drive/MyDrive/{downloadPath}" -o '{f["name"]}.%(ext)s' '{f["url"]}' --external-downloader aria2c --external-downloader-args "-c -j 8 -s 8 -x 8 -k 1M" --compat-options no-external-downloader-progress

downloadVideos()


To download from a torrent or magnet link

In [ ]:
import zipfile
import os
import libtorrent as lt
import time
import datetime
import shutil

params = {
    'save_path': '/content/Torrent/',
    'storage_mode': lt.storage_mode_t(2),
}

ses = lt.session()
ses.listen_on(6881, 6891)
link = input("Input Torrent file link or Magnet link and Press Enter: ")

if link.endswith('.torrent'):
    import wget
    from torf import Torrent

    if os.path.exists('torrent.torrent'):
        os.remove('torrent.torrent')

    wget.download(link, 'torrent.torrent')
    t = Torrent.read('torrent.torrent')
    link = str(t.magnet(name=True, size=False, trackers=False, tracker=False))


print(link)
handle = lt.add_magnet_uri(ses, link, params)
# change the 0 to a 1 to download sequentially
handle.set_sequential_download(0)
ses.start_dht()
begin = time.time()

print(datetime.datetime.now())
print('Downloading Metadata...')

while not handle.has_metadata():
    time.sleep(1)

print('Got Metadata, Starting Torrent Download...')
print("Starting", handle.name())

while handle.status().state != lt.torrent_status.seeding:
    s = handle.status()
    state_str = ['queued', 'checking', 'downloading metadata',
                 'downloading', 'finished', 'seeding', 'allocating']
    print('%.2f%% complete (down: %.1f kb/s up: %.1f kB/s peers: %d) %s ' %
          (s.progress * 100, s.download_rate / 1000, s.upload_rate / 1000,
           s.num_peers, state_str[s.state]))
    time.sleep(5)

end = time.time()
print(handle.name(), "COMPLETE")
print("Elapsed Time: ", int((end - begin) // 60), "min :", int((end - begin) % 60), "sec")
print(datetime.datetime.now())



if (input("Do you want to zip before uploading to drive? (Y/n): ").lower() == "y"):
  !zip -r /content/ /content/Torrent/*

Download Read-only Google Drive Files

In [ ]:
import os
from time import time
import gpsoauth
import requests
import json
from google.colab import userdata

currentTime = int(time())
driveToken = ''
validUntil = ''

def setToken():
  aasT = userdata.get('aasToken')
  androidID = userdata.get('androidID')
  email = userdata.get('email')
  auth = gpsoauth.perform_oauth(
    email,
    aasT,
    androidID,
    "oauth2:https://www.googleapis.com/auth/drive",
    app = "com.google.android.apps.docs",
    client_sig = "58e1c4133f7441ec3d2c270270a14802da47ba0e"
  )
  driveToken = os.environ["driveToken"] = auth["Auth"]
  validUntil = os.environ["validUntil"] = auth["Expiry"]

try:
  driveToken = os.environ["driveToken"]
  validUntil = os.environ["validUntil"]
  if int(validUntil) < currentTime:
    getFile()
  else:
    setToken()
except:
  setToken()

fileID = input("Enter file ID:")

def getFile(ID, token):
  uri = "https://www.googleapis.com/download/drive/v2internal/files/{}?alt=media".format(ID)
  headers = {"Authorization": "Bearer {}".format(token)}
  response = requests.get(uri, headers=headers)
  return response

def getFileMetadata(ID, token):
  uri = "https://www.googleapis.com/drive/v2internal/files/{}?supportsAllDrives=true&includePermissionsForView=published&allProperties=false&fields=publishingInfo%28published%29%2cmimeType%2cexportLinks%2cdownloadUrl%2ckind%2cfolderColorRgb%2csharedWithMeDate%2clastViewedByMeDate%2cpermissionsSummary%28visibility%28type%29%29%2ccontentRestrictions%2freadOnly%2cabuseIsAppealable%2cthumbnailVersion%2cheadRevisionId%2cmodifiedDate%2crecency%2cdriveId%2clabels%28starred%2cviewed%2crestricted%2ctrashed%29%2cparent%2fid%2ccreatedDate%2csubscribed%2calternateLink%2cid%2cversion%2cquotaBytesUsed%2cetag%2cdetectors%2cancestorHasAugmentedPermissions%2cfolderFeatures%2cspaces%2ccustomerId%2cabuseNoticeReason%2cworkspaceIds%2ctitle%2cshared%2chasAugmentedPermissions%2cparents%2fid%2cowners%28emailAddressFromAccount%2cid%2corganizationDisplayName%29%2ctrashedDate%2cresourceKey%2corganizationDisplayName%2copenWithLinks%2cdefaultOpenWithLink%2cfileSize%2chasLegacyBlobComments%2cexplicitlyTrashed%2creadersCanSeeComments%2clastModifyingUser%28id%2cemailAddressFromAccount%29%2ccapabilities%28canMoveItemWithinDrive%2ccanTrashChildren%2ccanRemoveChildren%2ccanReadCategoryMetadata%2ccanManageMembers%2ccanTrash%2ccanShare%2ccanAddMyDriveParent%2ccanListChildren%2ccanPrint%2ccanCopy%2ccanDeleteChildren%2ccanDelete%2ccanRename%2ccanModifyContent%2ccanRequestApproval%2ccanBlockOwner%2ccanCopyNonAuthoritative%2ccanReadDrive%2ccanMoveChildrenOutOfDrive%2ccanMoveItemOutOfDrive%2ccanDownload%2ccanShareChildFolders%2ccanChangePermissionExpiration%2ccanAddChildren%2ccanComment%2ccanAcceptOwnership%2ccanEdit%2ccanShareChildFiles%2ccanUntrash%2ccanManageVisitors%2ccanDownloadNonAuthoritative%2ccanChangeSecurityUpdateEnabled%2ccanReportSpamOrAbuse%2ccanMoveChildrenWithinDrive%29%2cactionItems%2cblockingDetectors%2cownedByMe%2cshortcutDetails%28targetResourceKey%2ctargetId%2ctargetMimeType%2ctargetLookupStatus%2ctargetFile%29%2cspamMetadata%28markedAsSpamDate%2cinSpamView%29%2cprimarySyncParentId%2cprimaryDomainName%2csharingUser%28emailAddressFromAccount%2cid%29%2cclientEncryptionDetails%28decryptionMetadata%29%2crecencyReason%2cmd5Checksum%2ccontainsUnsubscribedChildren%2capprovalMetadata%28approvalVersion%2capprovalSummaries%29%2chasThumbnail%2cmodifiedByMeDate%2cpassivelySubscribed&reportPermissionErrors=true&updateViewedDate=true&reason=1351&featureLabel=android-sync-native".format(ID)
  headers = {"Authorization": "Bearer {}".format(token)}
  response = requests.get(uri, headers=headers)
  return response.content

fileName = json.loads(getFileMetadata(fileID, driveToken))["title"]
print(fileName)
with open(fileName, "wb") as file:
    file.write(getFile(fileID, driveToken).content)

!mv "{fileName}" /content/drive/MyDrive/Blackbox/temp/

In [ ]:
from google.colab import drive
drive.mount('/content/drive')